In [45]:
import pandas as pd

movie_day14_df = pd.read_csv('./14data.csv', encoding='utf-8')

In [46]:
import requests
from bs4 import BeautifulSoup
import json
import random

page_type = "movie" #
search_type = "searchMovieInfo"
return_type = "json"

key_list=[
          'a0c311535c77ebdb3bab229d4955a54a',
          '797849ae4ee295a5b81f0b5628e37931',
          '206a3a64108e08359f4de77e77de1c4d',
          '0f56b7a37c358fa8fd15a4ffe5f78ab3',
          '16cc45cd8569a777e530e4c26e2c474f'
         ]
base = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/"

missing_movie_df = pd.DataFrame(columns=['movieNm','movie_code','nations','distribution','rate','genre','director','actor','cast'])
row = 0

for movie_code in movie_day14_df['movie_code']:
    try:
        row+=1
        param = "&movieCd=%s" % movie_code

        request_token = page_type + "/" + search_type + "." + return_type + "?key=" + random.choice(key_list)

        full_url = base + request_token+ param

        rep = requests.get(full_url)

        movie_data = rep.json()

        ##국적값 nations
        temp = []
        for item in movie_data["movieInfoResult"]["movieInfo"]["nations"]:
            temp.append(str(item.get("nationNm")))
            nations = "|".join(temp)

        ##장르값 genres
        temp = []
        for item in movie_data["movieInfoResult"]["movieInfo"]["genres"]:
            temp.append(str(item.get("genreNm")))
            genre = "|".join(temp)

        ##배우, 캐스트
        temp = []
        cast = []
        for item in movie_data["movieInfoResult"]["movieInfo"]["actors"]:
            temp.append(str(item.get("peopleNm")))
            cast.append(str(item.get("cast")))
            cast = list(filter(None, cast))
            actor_name = "|".join(temp)
            cast_name = "|".join(cast)

        ##감독값 director_name
        temp = []
        for item in movie_data["movieInfoResult"]["movieInfo"]["directors"]:
            temp.append(str(item.get("peopleNm")))
            director_name = "|".join(temp)

        temp = []
        for item in movie_data["movieInfoResult"]["movieInfo"]["companys"]:
            if str(item.get("companyPartNm")) == "배급사":
                temp.append(str(item.get("companyNm")))
            distribution = "|".join(temp)

        ##등급값 rate
        temp = []
        for item in movie_data["movieInfoResult"]["movieInfo"]["audits"]:
            temp.append(str(item.get("watchGradeNm")))
            rate = "|".join(temp)

        missing_movie_df.loc[row] = [movie_data["movieInfoResult"]["movieInfo"]["movieNm"],
                                     movie_code,
                                     nations,
                                     distribution,
                                     rate,
                                     genre,
                                     director_name,
                                     actor_name,
                                     cast_name
                                     ]
        print(row,end='\r')
    except:
        print(full_url)
        pass

http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=0f56b7a37c358fa8fd15a4ffe5f78ab3&movieCd=20164168


In [53]:
missing_movie_df.to_csv('./14_missing.csv', encoding='utf-8')

In [56]:
missing_movie_df = pd.read_csv('./14_missing.csv', encoding='utf-8')

In [57]:
%%time
new_df = 0
actor_filmo = pd.DataFrame(columns=("actor_name", "filmo","filmo_code"))

for index,row in missing_movie_df.iterrows():
    peopleNm_list = [] ### 매번 비우면서
    
    if '|' in getattr(row,"actor"):
        peopleNm_list = (getattr(row,"actor").split('|'))
    else:
        peopleNm_list.append(getattr(row,"actor"))
        
    filmoNames = getattr(row,"movieNm")
    filmocode = getattr(row,"movie_code")
    
    for actor in peopleNm_list:
        new_df+=1 ## actor_filmo row count
        actor_filmo.loc[new_df] = [actor,filmoNames,filmocode]

Wall time: 2min


In [58]:
actor_code_bk = pd.read_csv('./8actor_code_bak.csv', encoding='utf-8')

In [59]:
%%time
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd
import html5lib
import re
import time
import aiohttp
import asyncio
import lxml
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import asyncio

async def fetch(link,peopleNm,filmo,index):
    async with ClientSession() as session:
        async with session.get(link) as response:
            html = await response.read()
            try:
                soup = BeautifulSoup(html,"lxml")
                td = soup.find('td', class_='ellipsis')
                link = td.find('a')['onclick']
                code = int(re.search(r'\d+', link).group()) ## 코드(숫자)만 걸러내기
                total_actor_code_df.loc[index] = [peopleNm,filmo,code]
            except Exception as e: 
                pass
    
if __name__ == "__main__":
    count = 1
    length = 300
    tasks = []
    loop = asyncio.get_event_loop()
    
    total_actor_code_df = pd.DataFrame(columns=['actor_name', 'filmo','filmo_code','code'])
    
    for index, row in actor_filmo.iterrows():
        
        peopleNm = getattr(row,"actor_name")
        filmo = getattr(row,"filmo")
        filmocode = getattr(row,"filmo_code")
        
        if any(actor_code_bk['filmo'][actor_code_bk['actor_name']==peopleNm] == filmo):
            total_actor_code_df.loc[index] = [peopleNm,
                                              filmo,
                                              filmocode,
                                              actor_code_bk['code'][actor_code_bk['filmo']==filmo][actor_code_bk['actor_name']==peopleNm].item()
                                             ]
        else:
            peopleNm_parse = parse.quote(peopleNm)
            filmo_parse = parse.quote(filmo)

            param = "sPeopleNm=%s&sMovName=%s" % (peopleNm_parse,filmo_parse)

            url1 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleList.do?"

            link = url1 + param

            task = asyncio.ensure_future(fetch(link,peopleNm,filmo,index))
            tasks.append(task)

            if count == length: ## Task 300개 채우면
                print(index)
                loop.run_until_complete(asyncio.wait(tasks))
                tasks=[]
                length+= 300
                
            count+=1

        if index == len(actor_filmo): ## 나머지 Task 채우기
            loop.run_until_complete(asyncio.wait(tasks))
            tasks = []
            loop.close()
            

1706
3328
4802
6466
8083
9520
11033
12717
14382
15602
17100
Wall time: 12min 55s


In [60]:
total_actor_code_df.to_csv('./14act_code.txt', encoding='utf-8')

In [1]:
import pandas as pd
total_actor_code_df = pd.read_csv('./14act_code.txt')

In [31]:
%%time
import aiohttp
import asyncio
import lxml
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import requests

async def fetch(link,director_code,filmo_name,index,movie_code):
    async with ClientSession() as session:
        async with session.get(link) as response:
            html = await response.read()
            
            soup = BeautifulSoup(html,"lxml")

            tbody = soup.findAll('em', class_='tl')

            result = []

            for page in soup.find_all('p',class_='pageList pmt2'):
                page_num = list(page.strings)
                page_result = [i for i in page_num if i != '\n']
                
            for etcParam in page_result:
            
                url2 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleDtl.do?"

                param = "code=%s&sType=filmo&etcParam=%s"  % (director_code,etcParam)

                link = url2+param

                average = await get(link,filmo_name,result)
                                
                director_average_df.loc[index] = [director_code,
                                                  filmo_name,
                                                  movie_code,
                                                  average
                                                 ]
async def get(link,filmo_name,result):
    async with ClientSession() as session:
        async with session.get(link) as response:
            
            html = await response.read()
            
            soup = BeautifulSoup(html,"lxml")

            tr_tag = []
            real_tag = []
            for count in soup.findAll('dl'):
                if any(filmo_name in s for s in list(count.strings)):
                    pass
                elif any('주연' in x for x in list(count.strings)):
    #                 else:
                    tr_tag = tr_tag + list(count.strings)

            for elem in range(0, len(tr_tag)) :
                if tr_tag[elem] == '[공식통계]':
                    real_tag.append(tr_tag[elem+3])

            real_tag = [w.replace('명','') for w in real_tag] ## 명 없애기
            real_tag = [w.replace(',','') for w in real_tag] ## , 없애기
    #             real_tag = [w.replace('','0') for w in real_tag] ## , 없애기
            real_tag = list(filter(None, real_tag))    ## 빈값 지우기
            real_tag = list(map(int,real_tag))

            result.extend(real_tag)

            return int(average(result))

def average(values):
    if len(values) == 0:
        return 0
    return sum(values, 0.0) / len(values)
        
if __name__ == "__main__":
    count = 1
    length = 300
    tasks = []
    last = 0
    director_average_df = pd.DataFrame(columns=['code','filmo','movie_code','average_audience'])
    
    loop_director = asyncio.get_event_loop()
    
    for index,row in total_actor_code_df.iterrows():
        filmo_name = getattr(row,'filmo')
        director_code = getattr(row,'code')
        movie_code = getattr(row,'filmo_code')
        
        param = "code=%s&sType=filmo"  % (director_code)
        
        url1 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleDtl.do?"
        
        link = url1 + param
        
        task = asyncio.ensure_future(fetch(link,director_code,filmo_name,index,movie_code))
        tasks.append(task)
        
        if index == length:
            print(index)
            loop_director.run_until_complete(asyncio.wait(tasks))
            tasks = []
            length += 300
            
        if index == len(total_actor_code_df):
            loop_director.run_until_complete(asyncio.wait(tasks))
            tasks = []
            loop_director.close()


300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
7800
8100
8400
8700
9000
9300
9600
9900
10200
10500
10800
11100
11400
11700
12000
12300
12600
12900
13200
13500
13800
14100
14400
Wall time: 1h 30min 16s


In [32]:
director_average_df.to_csv('./14actor_average.csv', encoding='euc-kr')

In [45]:
import pandas as pd

movie_day14_df = pd.read_csv('./14data.csv', encoding='utf-8')

final_df = pd.read_csv('./14actor_average.csv', encoding='euc-kr')

missing_movie_df = pd.read_csv('./14_missing.csv')

actor_sum = pd.DataFrame(columns=['movie_name',
                                  'nation',
                                  'distribution',
                                  'director_average',
                                  'actor_sum',
                                  'rate',
                                  'genre',
                                  'release_date',
                                  'day1_screen',
                                  'day1_audience',
                                  'day14_date',
                                  'day14_audience'
                                 ])

for index,row in movie_day14_df.iterrows():
    movie_name = getattr(row, 'movie_name')
    movie_code = getattr(row, 'movie_code')
    director_average = getattr(row,'director_average')
    nation = getattr(row,'nation')
    release_date = getattr(row,'release_date')
    day1_screen = getattr(row,'day1_screen')
    day1_audience = getattr(row,'day1_audience')
    day15_date = getattr(row,'day14_date')
    day15_audience = getattr(row,'day14_audience')
    
    movie_name = movie_name[1:]

    try:
        actor_sum.loc[index] = [
                                movie_name,#                                 'movie_name',
                                nation,#                                   'nation',
                                missing_movie_df['distribution'][missing_movie_df['movie_code'] == movie_code].item(), ## 배급사
                                director_average,#                                   'director_sum',
                                sum(final_df['average_audience'][final_df['movie_code'] == movie_code]),#                 'actor_sum'
                                missing_movie_df['rate'][missing_movie_df['movie_code'] == movie_code].item(),#         'rate',
                                missing_movie_df['genre'][missing_movie_df['movie_code'] == movie_code].item(),#        'genre',
                                release_date,#        'release_date'
                                day1_screen,#          'day1_screen',
                                day1_audience, # day1_audience
                                day15_date,
                                day15_audience
                               ]
    except Exception as e:
        print(movie_name,index)
        print(e)

정글번치 : 빙산으로의 귀환 215
can only convert an array of size 1 to a Python scalar
정글번치 : 빙산으로의 귀환 220
can only convert an array of size 1 to a Python scalar
벤다 빌릴리 223
can only convert an array of size 1 to a Python scalar
벤다 빌릴리 271
can only convert an array of size 1 to a Python scalar
슈퍼미니 611
can only convert an array of size 1 to a Python scalar
슈퍼미니 620
can only convert an array of size 1 to a Python scalar
드래곤 기사단 637
can only convert an array of size 1 to a Python scalar
드래곤 기사단 713
can only convert an array of size 1 to a Python scalar
숲의 전설 787
can only convert an array of size 1 to a Python scalar
숲의 전설 791
can only convert an array of size 1 to a Python scalar
지휘자를 위한 1분 1023
can only convert an array of size 1 to a Python scalar
뉴타입 히어로 얏타맨 1037
can only convert an array of size 1 to a Python scalar
지휘자를 위한 1분 1057
can only convert an array of size 1 to a Python scalar
톰과 제리 아카데미 가다 1071
can only convert an array of size 1 to a Python scalar
뉴타입 히어로 얏타맨 1089
can only convert an a

In [43]:
missing_movie_df['distribution'][missing_movie_df['movie_code'] == movie_code]

1     프리비젼엔터테인먼트
62    프리비젼엔터테인먼트
Name: distribution, dtype: object

In [49]:
actor_sum.to_csv('./14_final.csv')